<a href="https://colab.research.google.com/github/emirunlu26/future-context-ST/blob/main/code/future_context_ST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

**READ THE REFERENCE TRANSCRIPTIONS OF EUROPARL**

In [ ]:
num_of_samples = 500

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/segments.en","r") as transcr_file:
  ref_transcriptions = list()
  for transcription in transcr_file:
    transcription = transcription[:len(transcription)-1]
    ref_transcriptions.append(transcription)
    if len(ref_transcriptions) == num_of_samples:
      break

In [ ]:
total_predictions = 0

for ref_sentence in ref_transcriptions:
  words_in_ref = ref_sentence.split()
  num_of_words_in_ref = len(words_in_ref)
  total_predictions += num_of_words_in_ref - 1

print(total_predictions)

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/Whisper-Wait-k/asr_target_test_set.txt","r") as transcr_file:
  ref_transcriptions = list()
  for transcription in transcr_file:
    transcription = transcription[:len(transcription)-1]
    ref_transcriptions.append(transcription)

In [ ]:
# COUNT NUMBER OF SENTENCES GROUPED BY NUMBER OF WORDS

def get_group_index(num_of_words):
  INTERVAL_SIZE = 5

  div = int(num_of_words / INTERVAL_SIZE)
  modulo = num_of_words % INTERVAL_SIZE

  if modulo == 0:
    return div - 1
  else:
    return div

intervals = [5,10,15,20,25,30,35,40,45,50,55,60,65]
size_of_intervals = [0,0,0,0,0,0,0,0,0,0,0,0,0]

for ref_sentence in ref_transcriptions:
  words_in_ref = ref_sentence.split()
  num_of_words_in_ref = len(words_in_ref)
  group_index = get_group_index(num_of_words_in_ref)
  size_of_intervals[group_index] += 1

start_number = 1
INTERVAL_SIZE = 5
for index, interval_size in enumerate(size_of_intervals):
  interval_name = str(start_number + index * INTERVAL_SIZE) + "-" + str(start_number + (((index+1) * INTERVAL_SIZE)-1))
  print("Interval " + interval_name + ": " + str(interval_size))

In [ ]:
word_predictions_sum = 0
for ref_sentence in ref_transcriptions:
  words_in_ref = ref_sentence.split()
  num_of_words_in_ref = len(words_in_ref)
  num_of_word_predictions = num_of_words_in_ref - 1
  word_predictions_sum += num_of_word_predictions

print("Total Word Predictions:", word_predictions_sum)

**DEFINE CLASS FOR FUTURE CONTEXT PREDICTION**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

class QwenFutureContextPredictor():

  def __init__(self, model_name, device, system_prompt, is_instruction_tuned, instruction_prefix = None, examples = None):
    self.model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map=device)
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.system_prompt = system_prompt
    self.is_instruction_tuned = is_instruction_tuned
    self.instruction_prefix = instruction_prefix
    self.examples = examples

  def predict(self, partial_text, max_new_tokens = 1, with_examples = False):
    user_prompt = self.instruction_prefix + " " + partial_text if self.is_instruction_tuned else partial_text
    model_inputs = self.tokenizer([user_prompt], return_tensors="pt").to(self.model.device)

    output_dict = self.model.generate(
    **model_inputs,
    max_new_tokens=max_new_tokens,
    return_dict_in_generate=True,
    output_scores=True
    )

    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, output_dict["sequences"])]
    prediction = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return (prediction,output_dict)


In [ ]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", torch_dtype="auto", device_map="cuda")

In [ ]:
txt = "This can be incomprhensible for"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

output = model.generate(
    **tokenizer([txt], return_tensors="pt").to(model.device),
    max_new_tokens=1,
    return_dict_in_generate=True,
    output_scores=True,
)

In [ ]:
print(tokenizer.batch_decode(output['sequences']), output['sequences'])

In [ ]:
for seq in output['sequences']:
  print("Tensor:", seq)
  print(tokenizer.batch_decode(seq))

In [ ]:
tokenizer.encode(' is')

In [ ]:
import torch

torch.exp(torch.nn.functional.log_softmax(output["scores"][0][0], dim=-1)[374])

**USE QWEN 2.5-0.5B-Instruct TO IMPLEMENT NEXT-WORD PREDICTION**

**FUTURE CONTEXT PREDICTION WITHOUT PREVIOUS CONTEXT**

In [ ]:
examples = [
    ("You must answer the European citizens as to why", "they"),
    ("I entirely agree", "with"),
    ("There cannot be a return to the status quo ante, and I", "think"),
    ("The Commission came up with a very carefully crafted position on third countries investing in the EU, suggesting", "that"),
    ("The answer, we are repeatedly told,", "is"),
]

# INSTANTIATE AN OBJECT FROM QwenFutureContextPredictor CLASS
qwen_predictor = QwenFutureContextPredictor(
    model_name= "Qwen/Qwen2.5-0.5B-Instruct",
    device= "cuda",
    system_prompt= "Your mission is the next-token prediction for a given incomplete sentence.",
    is_instruction_tuned= True,
    instruction_prefix= "Predict the next token for the incomplete sentence:",
    examples = examples
)

In [ ]:
example_input = "My dad wants me to"
prediction,output_dict = qwen_predictor.predict(example_input, max_new_tokens=1, with_examples=True)
print("\nPrediction:",prediction)

In [ ]:
input = qwen_predictor.tokenizer([ref_transcriptions[0]], return_tensors="pt").to(qwen_predictor.model.device)
print(input["input_ids"][0])
token_num = len(input["input_ids"][0])
print("Token num:", token_num)
num_of_tokens_to_read = round(token_num * 0.5)
print(input["input_ids"][0][:num_of_tokens_to_read])

partial_sentence = qwen_predictor.tokenizer.decode(input["input_ids"][0][:num_of_tokens_to_read + 1])
print("Partial sentence:", partial_sentence)

In [ ]:
# Take a specified amount of words from reference sentences as input at each step

initial_word_amount_to_read = 5 # amount of words in the input for the first step
last_word_amount_to_read = 25  # amount of words in the input for the last step
word_amount_step = 5  # number by which the amount of words in the input increases
num_of_tokens_to_predict = 8 # number of tokens which should be predicted by the model
with_examples = True

final_result_list = list() # store results of reference sentences as a list of list where each element is a list of results for a specific reference sentence

for ref_sentence in ref_transcriptions:
  result_list = list() # store each result for a specific reference sentence as elements of a list
  word_amount_to_read = initial_word_amount_to_read

  while word_amount_to_read <= last_word_amount_to_read:
    input_sentence = " ".join(ref_sentence.split()[:word_amount_to_read])
    print("Input sentence:", input_sentence)
    prediction, output_dict = qwen_predictor.predict(input_sentence, max_new_tokens=num_of_tokens_to_predict, with_examples=with_examples)
    predicted_words = prediction.split()
    print("Predicted text:", prediction)
    predicted_token_ids = qwen_predictor.tokenizer([prediction], return_tensors="pt").to(qwen_predictor.model.device)["input_ids"][0]
    result_dict = {
        "word_amount_to_read": word_amount_to_read,
        "predicted_token_ids": predicted_token_ids,
        "predicted_words": predicted_words,
        "input": input_sentence,
        "ground-truth_words": ref_sentence.split()[word_amount_to_read:word_amount_to_read + len(predicted_words)],
        "output_scores": output_dict["scores"]
        }

    print("Result:", result_dict)
    print()
    result_list.append(result_dict)
    word_amount_to_read += word_amount_step

  final_result_list.append(result_list)



**FUTURE CONTEXT PREDICTION WITH PREVIOUS CONTEXT**

In [ ]:
examples = [
    ("You must answer the European citizens as to why", "they"),
    ("I entirely agree", "with"),
    ("There cannot be a return to the status quo ante, and I", "think"),
    ("The Commission came up with a very carefully crafted position on third countries investing in the EU, suggesting", "that"),
    ("The answer, we are repeatedly told,", "is"),
]

# INSTANTIATE AN OBJECT FROM FutureContextPredictor CLASS
qwen_predictor = QwenFutureContextPredictor(
    model_name= "Qwen/Qwen2.5-0.5B-Instruct",
    device= "cuda",
    is_instruction_tuned = True,
    instruction_prefix= "Predict the next token for the given incomplete text:",
    system_prompt= "Your mission is the next-token prediction for a given incomplete sentence.",
    examples = examples
)

**GIVE EXAMPLES TO THE MODEL**

In [ ]:
def predict(qwen_predictor, input_sentence, previous_context, num_of_tokens_to_predict):
  if previous_context == "":
    return qwen_predictor.predict(input_sentence, max_new_tokens=num_of_tokens_to_predict, with_examples = True)
  else:
    return qwen_predictor.predict(previous_context + " " + input_sentence, max_new_tokens=num_of_tokens_to_predict, with_examples = True)

**DO NOT GIVE ANY EXAMPLES TO THE MODEL**

In [ ]:
def predict(qwen_predictor, input_sentence, previous_context, num_of_tokens_to_predict):
  if previous_context == "":
    return qwen_predictor.predict(input_sentence, max_new_tokens=num_of_tokens_to_predict, with_examples = False)
  else:
    return qwen_predictor.predict(previous_context + " " + input_sentence, max_new_tokens=num_of_tokens_to_predict, with_examples = False)

**GENERATE PREDICTIONS**

In [ ]:
# Take a specified amount of words from reference sentences as input at each step

initial_word_amount_to_read = 30 # amount of words in the input for the first step
last_word_amount_to_read = 30  # amount of words in the input for the last step
word_amount_step = 5  # number by which the amount of words in the input increases
num_of_tokens_to_predict = 4 # number of tokens which should be predicted by the model
previous_context_max_size = 10 # maximum amount of previous sentences

final_result_list = list() # store results of reference sentences as a list of list where each element is a list of results for a specific reference sentence

for index, ref_sentence in enumerate(ref_transcriptions):
  result_list = list() # store each result for a specific reference sentence as elements of a list
  word_amount_to_read = initial_word_amount_to_read

  while word_amount_to_read <= last_word_amount_to_read:
    input_sentence = " ".join(ref_sentence.split()[:word_amount_to_read])
    print("Input sentence:", input_sentence)
    previous_context_start_index = 0 if (index < previous_context_max_size) else (index - previous_context_max_size)
    previous_context = " ".join(ref_transcriptions[previous_context_start_index:index])
    prediction,output_dict = predict(qwen_predictor, input_sentence, previous_context, num_of_tokens_to_predict)
    predicted_words = prediction.split()
    predicted_token_ids = qwen_predictor.tokenizer([prediction], return_tensors="pt").to(qwen_predictor.model.device)["input_ids"][0]
    result_dict = {"word_amount_to_read": word_amount_to_read,
                   "predicted_token_ids": predicted_token_ids,
                   "predicted_words": prediction.split(),
                   "input": previous_context + " " + input_sentence,
                   "ground-truth_words": ref_sentence.split()[word_amount_to_read:word_amount_to_read + len(predicted_words)],
                   "output_scores": output_dict["scores"]
                   }
    print("Result:", result_dict)
    print()
    result_list.append(result_dict)
    word_amount_to_read += word_amount_step

  final_result_list.append(result_list)
print(final_result_list)

**USE QWEN 2.5-0.5B BASE MODEL TO IMPLEMENT NEXT-WORD PREDICTION**

In [ ]:
# INSTANTIATE AN OBJECT FROM QwenFutureContextPredictor CLASS
qwen_predictor = QwenFutureContextPredictor(
    model_name= "Qwen/Qwen2.5-0.5B",
    device= "cuda",
    system_prompt= "You are Qwen, created by Alibaba Cloud. You are a helpful assistant.",
    is_instruction_tuned= False
    )

In [ ]:
example_input = "I need to get some fresh hair, thus I will"
print(example_input)
output,output_dict = qwen_predictor.predict(example_input, max_new_tokens =3, with_examples=False)

print("******************")
print("Output:", output)
print("******************")

In [ ]:
# Take a specified amount of words from reference sentences as input at each step

initial_word_amount_to_read = 30 # amount of words in the input for the first step
last_word_amount_to_read = 30  # amount of words in the input for the last step
word_amount_step = 5  # number by which the amount of words in the input increases
num_of_tokens_to_predict = 4 # number of tokens which should be predicted by the model
with_examples = False

final_result_list = list() # store results of reference sentences as a list of list where each element is a list of results for a specific reference sentence

for ref_sentence in ref_transcriptions:
  result_list = list() # store each result for a specific reference sentence as elements of a list
  word_amount_to_read = initial_word_amount_to_read

  while word_amount_to_read <= last_word_amount_to_read:
    input_sentence = " ".join(ref_sentence.split()[:word_amount_to_read])
    print("Input sentence:", input_sentence)
    prediction,output_dict = qwen_predictor.predict(input_sentence, max_new_tokens=num_of_tokens_to_predict, with_examples=with_examples)
    predicted_words = prediction.split()
    print("Predicted text:", prediction)
    predicted_token_ids = qwen_predictor.tokenizer([prediction], return_tensors="pt").to(qwen_predictor.model.device)["input_ids"][0]
    result_dict = {
        "word_amount_to_read": word_amount_to_read,
        "predicted_token_ids": predicted_token_ids,
        "predicted_words": predicted_words,
        "input": input_sentence,
        "ground-truth_words": ref_sentence.split()[word_amount_to_read:word_amount_to_read + len(predicted_words)],
        "output_scores": output_dict["scores"]
        }
    print("Result:", result_dict)
    print()
    result_list.append(result_dict)
    word_amount_to_read += word_amount_step

  final_result_list.append(result_list)

**CALCULATE ACCURACY WORD BY WORD - START**

In [ ]:
import torch

def find_word_end_indexes(generated_tokens, num_of_words):
  word_end_indexes = list()
  token_index = 1
  for index in range(num_of_words):
    print("\nWord Number:", index+1)
    word_ended = False
    while (not word_ended) and token_index < len(generated_tokens):
      token = generated_tokens[token_index]
      print("Token:", token)
      if token[0] == " ":
        word_ended = True
      token_index += 1
    if word_ended:
      word_end_index = token_index - 2
    else:
      word_end_index = None
    word_end_indexes.append(word_end_index)
  print("Word End Indexes:", word_end_indexes)
  return word_end_indexes

def find_probability_of_words(word_end_indexes, generated_ids, output_dict):
  word_probabilities = list()
  start_index = 0
  for end_index in word_end_indexes:
    if end_index == None:
      word_probabilities.append(0)
      # no need to assign new value for start_index
      continue
    print("\nStart Index:", start_index)
    token_probability_sum = 0
    for token_index in range(start_index,end_index+1):
      print("Token Index:", token_index)
      token_id = generated_ids[token_index]
      print("Token id:", token_id)
      token_probability = torch.exp(torch.nn.functional.log_softmax(output_dict["scores"][token_index][0], dim=-1)[token_id])
      print("Token_probabilty:", token_probability)
      token_probability_sum += token_probability
    token_num = end_index - start_index + 1
    print("Token Number:", token_num)
    print("Token Probability Sum:", token_probability_sum)
    word_probability = token_probability_sum/token_num
    word_probabilities.append(word_probability)
    print("Word Probability:", word_probability)
    start_index = end_index + 1
  return word_probabilities

def predict(input_text, num_of_words_to_predict, previous_context, model, tokenizer, tokens_per_word = 4):
  max_new_tokens = num_of_words_to_predict * tokens_per_word
  if previous_context:
    input = previous_context + " " + input_text
  else:
    input = input_text
  print("Input to model:", input)
  model_inputs = tokenizer([input], return_tensors="pt").to(model.device)

  output_dict = model.generate(
      **model_inputs,
      max_new_tokens=max_new_tokens,
      return_dict_in_generate=True,
      output_scores=True
      )
  generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, output_dict["sequences"])]
  prediction = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  generated_tokens= tokenizer.batch_decode(generated_ids[0])
  print("Generated Tokens:", generated_tokens)
  word_end_indexes = find_word_end_indexes(generated_tokens, num_of_words_to_predict)
  word_probabilities = find_probability_of_words(word_end_indexes, generated_ids[0], output_dict)

  print("*******************************************************************")
  return (prediction,word_probabilities)



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B", torch_dtype="auto", device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
previous_context_max_size = 0 # maximum amount of previous sentences
correct_prediction_counter_by_threshold = {"0.0":0,"0.4":0, "0.5":0, "0.6":0, "0.7":0, "0.8":0}
total_prediction_counter_by_threshold = {"0.0":0,"0.4":0, "0.5":0, "0.6":0, "0.7":0, "0.8":0}
num_of_words_to_predict = 1
thresholds = [0.0,0.4, 0.5, 0.6, 0.7, 0.8]

for sentence_index,ref_sentence in enumerate(ref_transcriptions):
  previous_context_start_index = 0 if (sentence_index <= previous_context_max_size) else (sentence_index - previous_context_max_size)
  previous_context_sentences = ref_transcriptions[previous_context_start_index:sentence_index]
  previous_context = " ".join(previous_context_sentences)
  print("\nSentence", sentence_index+1)
  print("Previous context:", previous_context)
  print("Previous context length:", len(previous_context_sentences))
  words_in_ref = ref_sentence.split()
  for word_index in range(len(words_in_ref)-1):
    next_word_index = word_index + 1
    ground_truth_next_word = words_in_ref[next_word_index]
    input_text = " ".join(words_in_ref[:next_word_index])
    print("Input text:", input_text)
    prediction,word_probabilities = predict(input_text=input_text,
                                     num_of_words_to_predict=num_of_words_to_predict,
                                     previous_context=previous_context,
                                     model=model,
                                     tokenizer=tokenizer
                                     )
    predicted_next_word = prediction.split()[0]
    word_probability = word_probabilities[0]
    print("Word Probability:", word_probability)
    print("Ground truth next word:", ground_truth_next_word)
    print("Predicted next word:", predicted_next_word)

    # TO-DO: BİRDEN FAZLA THRESHOLD İÇİN KONTROL YAP
    for threshold in thresholds:
      if word_probability > threshold:
        total_prediction_counter_by_threshold[str(threshold)] += 1
        print("Hit?:", predicted_next_word == ground_truth_next_word)
        if predicted_next_word == ground_truth_next_word:
          correct_prediction_counter_by_threshold[str(threshold)] += 1
    print("\n***************************************************")

accuracy_by_threshold = {}
for threshold in thresholds:
  accuracy = round(correct_prediction_counter_by_threshold[str(threshold)]/total_prediction_counter_by_threshold[str(threshold)],3) * 100
  accuracy_by_threshold[str(threshold)] = accuracy



In [ ]:
for threshold in thresholds:
  print("Threshold (%):", threshold * 100)
  print("Correct predictions:", correct_prediction_counter_by_threshold[str(threshold)])
  print("Total predictions:", total_prediction_counter_by_threshold[str(threshold)])
  print("Accuracy:", accuracy_by_threshold[str(threshold)])
  print("\n\n")

**CALCULATE ACCURACY WORD BY WORD - END**

**USE Llama-3.2-1B BASE MODEL TO IMPLEMENT FUTURE-CONTEXT-PREDICTION**

In [ ]:
!huggingface-cli login

In [ ]:
import torch
from transformers import pipeline

class LlamaFutureContextPredictor():

  def __init__(self, model_id, device):
    self.pipe = pipeline(
        "text-generation",
        model=model_id,
        torch_dtype=torch.float16,
        device_map=device,
    )

  def predict(self, partial_text):
    return self.pipe(partial_text)

In [ ]:
llama_predictor = LlamaFutureContextPredictor(
    model_id = "meta-llama/Llama-3.2-1B",
    device = "cuda"
)

In [ ]:
partial_text = "Today is a beatiful"
print(llama_predictor.predict(partial_text)

**STORE OUTPUTS**

In [ ]:
output_lines = list()

for index_1, final_result in enumerate(final_result_list):
  output_line = "Sentence " + str(index_1 + 1) + "\n"
  for index_2, result in enumerate(final_result):
    output_line += "\nWord Amount to Read: " + str(result["word_amount_to_read"]) + "\n"
    output_line += "Input: " + result["input"] + "\n"
    output_line += "Prediction: " + " ".join(result["predicted_words"]) + "\n"
    output_line += "Ground-truth: " + " ".join(result["ground-truth_words"]) + "\n"
  output_line += "\n***************************************************************\n"
  output_lines.append(output_line)

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/future-context prediction/output.txt", "w") as file:
  file.writelines(output_lines)

**EVALUATE FUTURE CONTEXT PREDICTION**

In [ ]:
import numpy as np
import statistics
import torch

def get_results_for_step(final_result_list, step):
  return list(map(lambda x: x[step], final_result_list))

def prediction_matches_ground_truth(predicted_words, ground_truth_words):
  predicted_words_to_check = predicted_words[:len(ground_truth_words)]
  print("Predicted words to check:", predicted_words_to_check)
  print("Ground truth wordsto check:", ground_truth_words)
  return all(pred_and_ground[0] == pred_and_ground[1] for pred_and_ground in zip(predicted_words_to_check, ground_truth_words))

'''
Calculates the percentage of correct step results.
A step result for a sentence is correct only if all predicted tokens are correct.
'''
def calculate_success_rate(num_of_words_to_check, step_results, ref_transcriptions, tokenizer, predictor):
  step_results_to_check_counter = 0
  correct_step_results_num = 0
  pred_probabilities_for_correct_results = list()
  pred_probabilities_for_false_results = list()

  for step_result, ref_sentence in zip(step_results, ref_transcriptions):
    predicted_words_to_check = step_result["predicted_words"][:num_of_words_to_check]
    predicted_token_ids = step_result["predicted_token_ids"]
    predicted_tokens_num = len(predicted_token_ids)
    ground_truth_words_to_check = step_result["ground-truth_words"][:num_of_words_to_check]
    #ground_truth_tokens = tokenizer([ref_sentence], return_tensors="pt").to(predictor.model.device)["input_ids"][0]
    #ground_truth_next_token_ids = ground_truth_tokens[num_of_tokens_to_read:(num_of_tokens_to_read + predicted_tokens_num)]
    print("Predicted tokens:", tokenizer.batch_decode(predicted_token_ids))
    print("Predicted token ids:", predicted_token_ids)
    print("Number of predicted token ids:", len(predicted_token_ids))
    print("Reference sentence:", ref_sentence)
    #print("Number of tokens in reference:", len(ground_truth_tokens))
    #print("Number of tokens read:", num_of_tokens_to_read)
    #print("Ground truth tokens:", tokenizer.batch_decode(ground_truth_tokens,skip_special_tokens=True))
    #print("Ground truth next tokens:", tokenizer.batch_decode(ground_truth_next_token_ids))
    #print("Ground truth next token ids:", ground_truth_next_token_ids)

    is_to_be_checked = ground_truth_words_to_check != []

    if is_to_be_checked:
      step_results_to_check_counter += 1
      pred_probabilities = [torch.exp(torch.nn.functional.log_softmax(step_result["output_scores"][0][0], dim=-1)[predicted_token_ids[i]]) for i in range(predicted_tokens_num)]

    if prediction_matches_ground_truth(predicted_words_to_check, ground_truth_words_to_check) and is_to_be_checked:
      pred_probabilities_for_correct_results.append(pred_probabilities)
      correct_step_results_num += 1
      print("Hit")
    elif is_to_be_checked:
      pred_probabilities_for_false_results.append(pred_probabilities)
      print("Miss")
    print()

  print(len(pred_probabilities_for_correct_results) == correct_step_results_num)
  print("Correct results:", correct_step_results_num)
  print("Step results to check:", step_results_to_check_counter)

  pred_probabilities = {
      "for_correct": pred_probabilities_for_correct_results,
      "for_false": pred_probabilities_for_false_results
  }
  success_rate = round(correct_step_results_num / step_results_to_check_counter,3) * 100
  return (success_rate, pred_probabilities)

def get_pred_probability_means_for_index(predicted_token_index, pred_probabilities_for_step):
  pred_probabilities_for_correct_results = list(map(lambda x: x[predicted_token_index].item(), pred_probabilities_for_step["for_correct"]))
  pred_probabilities_for_false_results = list(map(lambda x: x[predicted_token_index].item(), pred_probabilities_for_step["for_false"]))

  return {
      "for_correct": round(statistics.mean(pred_probabilities_for_correct_results), 2),
      "for_false": round(statistics.mean(pred_probabilities_for_false_results), 2)
  }

predictor = qwen_predictor
tokenizer = predictor.tokenizer
predicted_token_index = 0
word_amount_to_read = initial_word_amount_to_read
step = 0
num_of_words_to_check = 1
success_rates_by_step = list()
pred_probability_results= {
    "predicted_token_index": predicted_token_index,
    "means_by_word_amount_to_read": {},
    }
while word_amount_to_read <= last_word_amount_to_read:
  print("****************************************************")
  print("Step " + str(step) + ", Word Amount to Read:", word_amount_to_read)
  print("****************************************************")
  step_results = get_results_for_step(final_result_list, step)
  print("Step results length:", len(step_results))
  success_rate, pred_probabilities_for_step = calculate_success_rate(num_of_words_to_check, step_results, ref_transcriptions, tokenizer, predictor)
  #pred_probability_means = get_pred_probability_means_for_index(predicted_token_index, pred_probabilities_for_step)
  print("Success rate for " + str(word_amount_to_read) + " of words in the reference sentences:", success_rate)
  print()
  success_rates_by_step.append(success_rate)
  #pred_probability_results["means_by_word_amount_to_read"][word_amount_to_read] = pred_probability_means
  step += 1
  word_amount_to_read += word_amount_step

In [ ]:
# Print Success Rate Results
for index, success_rate in enumerate(success_rates_by_step):
  word_amount_to_read = initial_word_amount_to_read + index * word_amount_step
  print("Success rate for first " + str(word_amount_to_read) + " from the reference sentence:", round(success_rate,2))

In [ ]:
# Print Prediction Probability Mean Results
means_by_percent = pred_probability_results["means_by_percent"]
print("Means of prediction probabilities for each step in % (predicted token index = " + str(pred_probability_results["predicted_token_index"]) + "):\n\n")
for index in range(len(success_rates_by_step)):
  percentage = start_percentage + index * percentage_step
  mean_for_correct_sentences = means_by_percent[percentage]["for_correct"] * 100
  mean_for_false_sentences = means_by_percent[percentage]["for_false"] * 100
  print("Prediction probability mean for correct sentences with " + str(round(percentage,2) * 100) + " % of reference sentences as input:", mean_for_correct_sentences)
  print("Prediction probability mean for correct sentences with " + str(round(percentage,2) * 100) + " % of reference sentences as input:", mean_for_false_sentences)
  print()

In [ ]:
# Print Prediction Probability Mean Results
means_by_percent = pred_probability_results["means_by_percent"]
print("Means of prediction probabilities for each step in % (predicted token index = " + str(pred_probability_results["predicted_token_index"]) + "):\n\n")
for index in range(len(success_rates_by_step)):
  percentage = start_percentage + index * percentage_step
  mean_for_correct_sentences = means_by_percent[percentage]["for_correct"] * 100
  mean_for_false_sentences = means_by_percent[percentage]["for_false"] * 100
  print("Prediction probability mean for correct sentences with " + str(round(percentage,2) * 100) + " % of reference sentences as input:", mean_for_correct_sentences)
  print("Prediction probability mean for correct sentences with " + str(round(percentage,2) * 100) + " % of reference sentences as input:", mean_for_false_sentences)
  print()